Final Project Check-in 2018-11-16
------

Group Name
-----

Student Names
----

1. 
2. 
3. 
4.  
5. 

Load Data
-----

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
data = pd.read_csv('~/desktop/finalproject/data.csv')
data = data[data.shot_made_flag.notnull()].reset_index()

features = ['combined_shot_type', 'period', 'playoffs', 'seconds_remaining', 'minutes_remaining', 'season',
          'shot_distance', 'shot_made_flag', 'shot_zone_area', 'game_date', 'matchup']

dataset = data[features]
dataset['total_seconds_remaining'] = 60*dataset['minutes_remaining'] + dataset['seconds_remaining']
dataset.drop('minutes_remaining', axis=1, inplace=True)
dataset.drop('seconds_remaining', axis=1, inplace=True)
dataset['month'] = dataset['game_date'].map(lambda x: x.split('-')[1])
dataset.drop('game_date', axis=1, inplace=True)
dataset['home'] = dataset['matchup'].str.contains('vs').astype('int')
dataset.drop('matchup', axis=1, inplace=True)
dataset["combined_shot_type"] = dataset["combined_shot_type"].astype('category')
dataset["period"] = dataset["period"].astype('category')
dataset["playoffs"] = dataset["playoffs"].astype('category')
dataset["season"] = dataset["season"].astype('category')
dataset["shot_made_flag"] = dataset["shot_made_flag"].astype('category')
dataset["month"] = dataset["month"].astype('category')
dataset["home"] = dataset["home"].astype('category')
multi_categorical_features = ['combined_shot_type', 'period', 'season', 'month', 'shot_zone_area']

for c in multi_categorical_features:
    dummies = pd.get_dummies(dataset[c], drop_first=True)
    dummies = dummies.add_prefix("{}#".format(c))
    dataset.drop(c, axis=1, inplace=True)
    dataset = dataset.join(dummies)

Fit scikit-learn model
----

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss


X = dataset.loc[:, dataset.columns != 'shot_made_flag']
Y = dataset['shot_made_flag']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

max_leaf_nodes = 10
dt = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes,
                            criterion='entropy',
                            random_state=42)
dt.fit(X_train, Y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

Evaluation Metric
----

In [13]:
node_count = dt.tree_.node_count
predicted = dt.predict(X_test)
print(f'Decision tree with {node_count} nodes')
print(f"Accuracy: {log_loss(Y_test, dt.predict_proba(X_test)):.3f}")

Decision tree with 19 nodes
Accuracy: 0.652
